In [2]:
import pandas as pd
import numpy as np
import os

# Path to original datasets
data_dir = "../dataset/original/"
sales_train = pd.read_csv(data_dir + 'sales_train_validation.csv')
sales_eval = pd.read_csv(data_dir + 'sales_train_evaluation.csv')
sell_prices = pd.read_csv(data_dir + 'sell_prices.csv')
calendar = pd.read_csv(data_dir + 'calendar.csv')

In [3]:
# Sum all unit sales per store
sales_train["total_sales"] = sales_train.iloc[:, 6:].sum(axis=1)  # d_1 to d_1941 columns
store_sales = sales_train.groupby("store_id")["total_sales"].sum().reset_index()

# Get store with highest total unit sales
top_store_id = store_sales.sort_values("total_sales", ascending=False).iloc[0][
    "store_id"
]

print(f"Store with highest total unit sales: {top_store_id}")

Store with highest total unit sales: CA_3


In [4]:
# Reduce dataframes to the store with the highest total unit sales
sales_train_top = sales_train[sales_train["store_id"] == top_store_id]
sales_eval_top = sales_eval[sales_eval["store_id"] == top_store_id]
sell_prices_top = sell_prices[sell_prices["store_id"] == top_store_id]

# Drop unnecessary columns: store_id, state_id, (item_id+store_id)
sales_train_top = sales_train_top.drop(columns=["store_id", "state_id", "id"])
sales_eval_top = sales_eval_top.drop(columns=["store_id", "state_id", "id"])
sell_prices_top = sell_prices_top.drop(columns=["store_id"])

In [5]:
calendar["date"] = pd.to_datetime(calendar["date"])
calendar.drop(columns=[col for col in calendar.columns if "snap" in col], errors="ignore", inplace=True)

## RAM 
keep_pct = 0.5  # keep last 50% of days


# Determine which days to keep
day_cols = [col for col in sales_train_top.columns if col.startswith("d_")]
num_keep = int(len(day_cols) * keep_pct)
keep_days = day_cols[-num_keep:]  # last N% of days

# Subset the sales data
sales_train_top = sales_train_top[["item_id", "dept_id", "cat_id"] + keep_days]

# Filter calendar and prices accordingly
calendar = calendar[calendar["d"].isin(keep_days)]
remaining_wm_yr_wks = calendar["wm_yr_wk"].unique()
sell_prices = sell_prices[sell_prices["wm_yr_wk"].isin(remaining_wm_yr_wks)]



In [7]:
# Save CSVs to /dataset/raw directory
out_dir = "../dataset/raw/"
os.makedirs(out_dir, exist_ok=True)
sales_train_top.to_csv(out_dir + "sales_train.csv", index=False)
sales_eval_top.to_csv(out_dir + "sales_eval.csv", index=False)
sell_prices_top.to_csv(out_dir + "sell_prices.csv", index=False)
calendar.to_csv(out_dir + "calendar.csv", index=False)

# Save the top store ID for reference
with open(out_dir + "top_store_id.txt", "w") as f:
    f.write(top_store_id)